# Extract airport and seaport codes from uktradeinfo.com Port_codes spreadsheet

> The air and seaport codes are for use when completing Customs documentation for UK trade with countries outside the European Union.

> The codes are to be used to complete the Location of Goods box (Box 30 of the SAD).

> There are two types of airports - Designated Airports and Non-designated. Non-designated Airports are those with a Certificate of Agreement (CoA) or a military base operating under a Memorandum of Understanding (MoU) or Agreement (MoA).

> Seaports in upper case are trade ports for which trade statistics are available. Ports in lower case are not identified separately for trade statistics purposes, but their trade is included in the statistics for the ports shown in upper case.


Load the Excel spreadsheet and store a copy locally

In [1]:
from databaker.framework import *
import requests
import json
from pathlib import Path
from collections import defaultdict
from string import capwords

ports_file = Path('in/Port_codes.xls')
if not (ports_file.exists() and ports_file.is_file()):
    response = requests.get('https://www.uktradeinfo.com/CodesAndGuides/Documents/Port_codes.xls')
    with open(ports_file, 'wb') as f:
        f.write(response.content)

sheets = loadxlstabs(ports_file)


Loading in/Port_codes.xls which has size 110080 bytes
Table names: ['Information', 'Airport codes', 'Seaport codes', 'Other codes']


The first tab/sheet is a description and the second tab should be named 'Airport codes'.

The 'alpha codes' are the International Air Transport Association (IATA) airport codes, which we'll use as the identity for each airport. We'll collect the rest of the information for each airport as is and output a JSON representation for further processing.

In [2]:
assert len(sheets) == 4
airport_codes = sheets[1]
assert airport_codes.name == 'Airport codes'

airports = [
    {
        '@id': airport.shift(RIGHT).value,
        '@type': airport.shift(RIGHT).shift(RIGHT).shift(RIGHT).value,
        'label': airport.value,
        'code': airport.shift(RIGHT).value,
        'seq': airport.shift(RIGHT).shift(RIGHT).value
    } for airport in airport_codes.excel_ref('A1').fill(DOWN)
]

print(json.dumps(airports, indent=2))

[
  {
    "@id": "ABZ",
    "@type": "Designated",
    "label": "Aberdeen Airport",
    "code": "ABZ",
    "seq": "449"
  },
  {
    "@id": "AOA",
    "@type": "Not applicable",
    "label": "All Other Airports",
    "code": "AOA",
    "seq": "481"
  },
  {
    "@id": "BFS",
    "@type": "Designated",
    "label": "Belfast International Airport",
    "code": "BFS",
    "seq": "427"
  },
  {
    "@id": "BQH",
    "@type": "Designated",
    "label": "Biggin Hill Airport",
    "code": "BQH",
    "seq": "474"
  },
  {
    "@id": "BHX",
    "@type": "Designated",
    "label": "Birmingham Airport",
    "code": "BHX",
    "seq": "433"
  },
  {
    "@id": "BLK",
    "@type": "Designated",
    "label": "Blackpool International Airport",
    "code": "BLK",
    "seq": "459"
  },
  {
    "@id": "BOH",
    "@type": "Designated",
    "label": "Bournemouth (Hurn) Airport",
    "code": "BOH",
    "seq": "441"
  },
  {
    "@id": "BRS",
    "@type": "Designated",
    "label": "Bristol Airport",
    "co

Put this into a JSON-LD structure and output.

In [3]:
airports_context = {
    "rdfs": "http://www.w3.org/2000/01/rdf-schema#",
    "schema": "http://schema.org/",
    "ports": "https://www.uktradeinfo.com/def/ports#",
    "iata": "schema:iataCode",
    "label": "rdfs:label",
    "seq": "ukti:sequence",
    "@base": "https://www.uktradeinfo.com/id/airports/",
    "Designated": "ports:DesignatedAirport",
    "Non-designated": "ports:NonDesignatedAirport",
    "Military": "ports:MilitaryAirport",
    "Not applicable": "ports:NotApplicable"
}

json_ld = {"@context": airports_context, "@graph": airports}
with open('out/airports.jsonld', 'w') as airports_file:
    json.dump(json_ld, airports_file, indent=2)

The next tab is for seaports. There's more information codified in this table, e.g. a label all in uppercase means that this is the canonical name for the various ports that the statistics will be grouped together into.

The JSON output captures this, using the alpha code as the primary ID and each record can then list the labels of the ports that are included in this group, if any.

In [4]:
seaport_codes = sheets[2]
assert seaport_codes.name == 'Seaport codes'

seaports = defaultdict(dict)
for seaport in seaport_codes.excel_ref('A1').fill(DOWN):
    label = seaport.value
    alpha = seaport.shift(RIGHT).value
    seq = seaport.shift(RIGHT).shift(RIGHT).value
    location = seaport.shift(RIGHT).shift(RIGHT).shift(RIGHT).value
    comment = seaport.shift(RIGHT).shift(RIGHT).shift(RIGHT).shift(RIGHT).value
    
    if 'seq' in seaports[alpha]:
        assert seaports[alpha]['seq'] == seq
    else:
        seaports[alpha]['seq'] = seq
    
    if location != 'Not applicable':
        if 'location' in seaports[alpha]:
            assert seaports[alpha]['location'] == location
        else:
            seaports[alpha]['location'] = location
    
    if comment != "":
        if 'comment' in seaports[alpha]:
            assert seaports[alpha]['comment'] == comment
        else:
            seaports[alpha]['comment'] = comment

    if label.isupper():
        seaports[alpha]['label'] = capwords(label)
    else:
        newport = {"@id": label.lower().replace(' ', '_'), "label": label, "@type": "seaport"}
        if 'includes' in seaports[alpha]:
            seaports[alpha]['includes'].append(newport)
        else:
            seaports[alpha]['includes'] = [newport]

seaports_list = [ {**v, '@id': k, '@type': 'trade'} for (k, v) in seaports.items() ]

print(json.dumps(seaports_list, indent=2))

[
  {
    "seq": "164",
    "location": "Scotland",
    "label": "Aberdeen",
    "@id": "ABD",
    "@type": "trade"
  },
  {
    "seq": "203",
    "location": "England & Wales",
    "includes": [
      {
        "@id": "althorpe",
        "label": "Althorpe",
        "@type": "seaport"
      },
      {
        "@id": "beckingham",
        "label": "Beckingham",
        "@type": "seaport"
      },
      {
        "@id": "burton_stather",
        "label": "Burton Stather",
        "@type": "seaport"
      },
      {
        "@id": "flixborough_wharf",
        "label": "Flixborough Wharf",
        "@type": "seaport"
      },
      {
        "@id": "gainsborough",
        "label": "Gainsborough",
        "@type": "seaport"
      },
      {
        "@id": "grove_wharves",
        "label": "Grove Wharves",
        "@type": "seaport"
      },
      {
        "@id": "guness_wharf",
        "label": "Guness Wharf",
        "@type": "seaport"
      },
      {
        "@id": "keadby",
        "la

Put this into JSON-LD.

In [5]:
seaports_context = {
    "rdfs": "http://www.w3.org/2000/01/rdf-schema#",
    "schema": "http://schema.org/",
    "ports": "https://www.uktradeinfo.com/def/ports#",
    "seq": "ukti:sequence",
    "@base": "https://www.uktradeinfo.com/id/seaports/",
    "includes": {"@reverse": "ports:includedIn"},
    "seaport": "ports:Seaport",
    "trade": "ports:TradeSeaport"
}

with open('out/seaports.jsonld', 'w') as seaport_file:
    json.dump({'@context': seaports_context, '@graph': seaports_list}, seaport_file, indent=2)
